In [61]:
import langchain
import os 
print(langchain.__version__)
from langsmith import traceable
from langsmith import traceable
from langchain_ollama import OllamaLLM
from langchain_core.runnables import RunnableLambda

0.3.1


In [24]:
import os
from dotenv import load_dotenv
load_dotenv()  # ✅ Load .env first

print(os.getenv("LANGSMITH_API_KEY")) 
print(os.getenv("LANGSMITH_PROJECT"))
print(os.getenv("LANGSMITH_TRACING"))


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
pr-enchanted-class-11
true


In [25]:
from dotenv import load_dotenv
load_dotenv()


True

In [26]:
import os
print("PROJECT:", os.getenv("LANGSMITH_PROJECT"))

PROJECT: pr-enchanted-class-11


In [128]:
from langsmith import traceable
from langchain_ollama import OllamaLLM
from langchain_core.runnables import RunnableLambda

@traceable
def main():
    llm = OllamaLLM(model="llama3", temperature=0)

    # Wrap LLM call in a RunnableLambda to enable tracing
    chain = RunnableLambda(lambda x: llm.invoke(x))

    result = chain.invoke("Tell me a joke about coffee ☕")
    print("LLM Output:", result)

main()


LLM Output: Why did the coffee file a police report?

Because it got mugged! 😄


In [129]:
import os
from dotenv import load_dotenv
from langsmith import traceable
from langchain_ollama import OllamaLLM

# Load environment variables
load_dotenv()

# Confirm they loaded
print("API_KEY:", os.getenv("LANGSMITH_API_KEY"))
print("PROJECT:", os.getenv("LANGSMITH_PROJECT"))
print("TRACING:", os.getenv("LANGSMITH_TRACING"))

@traceable
def main():
    llm = OllamaLLM(model="llama3", temperature=0)
    response = llm.invoke("tell me a joke")
    print("LLM Response:", response)

main()


API_KEY: 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
PROJECT: pr-enchanted-class-11
TRACING: true
LLM Response: Here's one:

Why couldn't the bicycle stand up by itself?

(Wait for it...)

Because it was two-tired!

Hope that made you smile! Do you want to hear another one?


In [ ]:
#llm = OllamaLLM(model="qwen:32b", temperature=0)
#response = llm.invoke("tell me a joke")
#print("LLM Response:", response)


KeyboardInterrupt: 

In [30]:
from langchain_core.output_parsers import StrOutputParser

OutputParser = StrOutputParser()
OutputParser.invoke(response)

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(Wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile! Do you want to hear another one?"

In [31]:
chain = llm | OutputParser
chain.invoke("tell me a joke")

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(Wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile! Do you want to hear another one?"

In [ ]:
from typing import List
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# Define Pydantic model
class MobileReview(BaseModel):
    phone_number: str = Field(description="Name and model of the phone")
    rating: float = Field(description="Overall rating out of 5")
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    summary: str = Field(description="Brief summary of the review")

# Review text
review_text = """
The iPhone 15 features a sleek design, the powerful A16 Bionic chip, a 48MP main camera, and introduces USB-C charging and the Dynamic Island for enhanced multitasking. 
It offers fast performance, improved photography, and better compatibility with accessories. However, it still lacks a 120Hz display, has no telephoto lens,
and uses an aluminum frame instead of more premium materials.
"""

# Define LLM

# Structured output setup
structure_llm = llm.with_structured_output(MobileReview)

# Get output
output = structure_llm.invoke(review_text)
print(output)


ImportError: cannot import name 'OllamaLLM' from 'langchain_community.llms' (/Users/nawafalhussain/anaconda3/envs/gpt_agetns/lib/python3.11/site-packages/langchain_community/llms/__init__.py)

In [38]:
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List

# 1. Define the schema
class MobileReview(BaseModel):
    phone_number: str = Field(description="Name and model of the phone")
    rating: float = Field(description="Overall rating out of 5")
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    summary: str = Field(description="Brief summary of the review")

# 2. Setup the output parser
parser = PydanticOutputParser(pydantic_object=MobileReview)

# 3. Create the prompt
prompt = PromptTemplate.from_template("""
Extract a structured mobile phone review from the following text.

Text:
{text}

{format_instructions}
""")

# 4. Combine prompt + LLM + parser into a chain
llm = OllamaLLM(model="llama3", temperature=0)
chain = prompt | llm | parser

# 5. Input review text
review_text = """
The iPhone 15 features a sleek design, the powerful A16 Bionic chip, a 48MP main camera, and introduces USB-C charging and the Dynamic Island for enhanced multitasking. 
It offers fast performance, improved photography, and better compatibility with accessories. However, it still lacks a 120Hz display, has no telephoto lens,
and uses an aluminum frame instead of more premium materials.
"""

# 6. Run the chain
output = chain.invoke({
    "text": review_text,
    "format_instructions": parser.get_format_instructions()
})

# 7. Print the structured result
output


MobileReview(phone_number='iPhone 15', rating=4.0, pros=['Sleek design', 'Powerful A16 Bionic chip', '48MP main camera', 'USB-C charging', 'Dynamic Island for enhanced multitasking', 'Fast performance', 'Improved photography', 'Better compatibility with accessories'], cons=['No 120Hz display', 'No telephoto lens', 'Aluminum frame instead of premium materials'], summary='The iPhone 15 offers a great balance of features and performance, but falls short in some areas.')

In [39]:
output.pros

['Sleek design',
 'Powerful A16 Bionic chip',
 '48MP main camera',
 'USB-C charging',
 'Dynamic Island for enhanced multitasking',
 'Fast performance',
 'Improved photography',
 'Better compatibility with accessories']

#LLM Message

In [93]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage,SystemMessage


System_Message=SystemMessage(content="you are helpful assistence that tells jokes.")
Human_message= HumanMessage(content="tell me about programming")
llm.invoke([System_Message,Human_message])

"Programming is the process of creating instructions for computers to follow. It involves writing code in a specific programming language, such as Python, Java, or C++.\n\nThe main goal of programming is to automate tasks, build software applications, create websites, and solve complex problems.\n\nHere's a brief overview of the programming process:\n\n1. **Problem definition**: Identify the task or problem you want to solve with programming.\n\n2. **Language selection**: Choose the appropriate programming language for your task.\n\n3. **Code writing**: Write the instructions in code form, following the syntax rules of the chosen language.\n\n4. **Testing and debugging**: Test your code to ensure it works as intended. Debug any errors or issues that arise during testing.\n\n5. **Deployment**: Once your code is tested and working correctly, you can deploy it to a server or hosting platform.\n\nRemember, programming is a continuous learning process, with new technologies and frameworks c

# RAG

In [49]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from typing import List
from langchain_core.documents import Document

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

# Load the .docx document
docx_loader = Docx2txtLoader("/Users/nawafalhussain/Desktop/multi_agent_hr/test/data/hr_policy.docx")
documents = docx_loader.load()

# Split the documents
splits = text_splitter.split_documents(documents)

# Print how many chunks were created
print(f"Split the documents into {len(splits)} chunks")


Split the documents into 137 chunks


In [72]:

from pdf2image import convert_from_path
import pytesseract

images = convert_from_path("/Users/nawafalhussain/Desktop/multi_agent_hr/test/data/hr_policy.pdf", dpi=300)

for i, img in enumerate(images):
    text = pytesseract.image_to_string(img, lang='ara')
    print(f"Page {i+1}:\n{text}")

splits = text_splitter.split_documents(documents)




python(86907) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
python(86908) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
python(86909) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forke

Page 1:
5أعططةوطء ذه
5 بام

يناير 2025م الإصدار الثالث

هذا الدليل قابل للتحديث بناءً على ما يطرأ من تغيير على سياسات الموارد البشرية




python(87109) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Page 2:
ك5أاع مط ةلا يه

© لإ

الباب الأول: الأحكام العامة والتعريفات غك
1. الأحكام العامة 44 111
2 التعريفات 0
الباب الثاني: تخطيط القوى العاملة والتوظيف 7
1 سياسة تخطيط القوى العاملة 0
1.1 تخطيط القوى العاملة 0
12 مخصصات التوظيف ا 0
13 نقل الموظفين ا
1.4 توظيف المقاولين 0
15 توظيف العامل الحر ا 0
16 الإحلال ممم ممم ممم مم ممه ممه ممه ممم مم ممه مم ممم ممه ممه ممم ممعم ممم ممع ممه ممعم مع ممه ممع مع عع مط مم6 2 110
2 سياسة التوظيف الداخلي 0
2.1 ضوابط التوظيف الداخلي للمستوى الإداري الخامس فقو مهمه ممه ممم ممم ممه ممم ممم ممه ممم ممم ممم مم6 2 10
22 شروط الترشح للتوظيف الداخلي ا 0
2.4 تحويل الوظيفة من داخلية إلى خارجية والعكس 0
25 العائد المادي 1-0 2 121212121212 12121 1 1 1 1 1 0
2.6 الإنتقال من مدينة إلى أخرى ا 0
257 التعيين أو التكليف للوظائف الإشرافية أو القيادية مممم ممه مم ممه مومه مهمه ممم مومهو مهمه مم مم ممم ومو ووو 2 12
3 سياسة الإستقطاب والتوظيف الخارجي 0
3.13 الفحص الطبي ا ا
327 تاريخ سريان العقد 01 ا
3218 إلغاء التعيين 0 طشط+<!|!|[|[ |[ 1 212121212121212 2 12 12121 1 1 1 1

python(87208) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Page 56:
ك5أاع مط ةلا يه
© لإ

الباب السابع: بيئة العمل

2 سياسة قواعد السلوك والجزاءات

تسعى الشركة إلى إيجاد بيئة عمل صحية تحافظ على سير العمل وعلى سمعتها وكرامة جميع موظفيها وسلامتهم مما يضر بهم جسدياً أو
معنويأ» وذلك من خلال تحديد واجبات الموظفين والشركة» وبيان المحظورات الواجب عليهم إجتنابهاء وفرض الجزاءات التأديبية عند مخالفة
ذلك

1 واجبات الموظفين

1 الترفع عن كل ما يخل بشرف الوظيفة والكرامة سواءً كان ذلك في مكان العمل أو خارجه.

2 مراعةة آداب اللباقة مع الجميع بلا إستثناء.

3 تخصيص وقت العمل لآداء واجبات الوظيفة وتنفيذ الأوامر بدقة وأمانة في حدود النظم والتعليمات.

4 حفظ أسرار العمل وعدم إفشائها بصورة مباشرة أو غير مباشرة.

5 التقيد بالتعليمات والأوامر المتعلقة بالعمل ما لم يكن فيها ما يخالف عقد العمل أو النظام أو الآداب العامة» ولم يكن في تنفيذها ما
556 مراعة تعليمات السلامة في آداء العمل والمحافظة على الأدوات المخصصه طبقاً للقواعد المحددة لها.

7 التعاون مع زملاء العمل على الوجه الذي يؤدي إلى حسن سير العمل بالشركة.

8 احترام مواعيد الحضور والانصراف والتواجد خلال ساعات العمل ف

python(90068) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Page 57:
ك5أاع مط ةلا يه
© لإ

2 إثارة الفوضى الجماعية في مقر العمل.
3 ممارسة العنصرية والإساءة إلى الآخرين قولاً أو فعلآ سواءً تصريحاً أو تلميحاً أو التقليل من شأن الآخرين بأي شكل من الأشكال
كبلد أو قبيلة أو دين.

23 واجبات الشركة تجاه الموظفين

1 معاملتهم بشكل لائق يبرز إهتمامها بأحوالهم ومصالحهم والإمتناع عن كل قول أو فعل يمس كرامتهم أو دينهم.

2 تعطي الشركة موظفيها الوقت اللازم لممارسة حقوقهم المنصوص عليها في نظام العمل ولائحة تنظيم العمل دون تخفيض لأجورهم
لقاء هذا الوقت» ويحق لها أن تنظم ممارسة هذا الحق بصورة لا تخل بسير العمل.

3 دفع أجور الموظفين وفق الضوابط المنظمة لذلك.

24 الجزاءات التأديبية

تنقسم الجزاءات التأديبية حسب لائحة الشركة إلى الأقسام التالية:

1 الإنذار.

2 حسم نسبة من الأجر في حدود جزء من الأجر اليومي.

3 الحسم الجزئي من الأجر الشهري و يتراوح بين أجر يوم و5 أيام.

4 الإيقاف عن العمل دون أجر.

5 تاأخير الترقية أو الحرمان من العلاوة الدورية لمدة لا تزيد على سنة واحدة.

6 االفصل من الخدمة مع المكافأة.

7 الفصل من الخدمة بدون مكافأة.

-57-



python(90119) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Page 58:
ك5أاع مط ةلا يه
© لإ

3. سياسة معالجة اعتراضات الموظفين
تكفل الشركة حق الإعتراض سعياً لتحقيق العدالة والمساواة المهنية بين الموظفين ولتهيئة بيئة عمل مناسبة وفق التسلسل الإداري وتطبيق
الأنظمة دون إستثناء أو تمييز.
4 ضوابط الاعتراضات
1 الإلتزام بإتباع الموظف للتسلسل الإداري.
2 أن يتم من خلال القناة المخصصة لذلك.
3 الإلتزام بالقواعد والضوابط والتعليمات التي تصدرها الشركة في هذا الشأن.
4 أن يتقدم الموظف بتظلمه أو شكواه على القرارات الإدارية وعلى نتائج تقييم الأداء خلال 5 أيام عمل من تاريخ علمه؛ أو تبليغه
بنتائج التقييم» أو القرار محل التظلمء أو الشكوى.
5 يخطر الموظف بنتيجة البت في التظلم خلال مدة لا تتجاوز 10 أيام عمل من تاريخ التظلم أو الشكوى.
6 أن يكون التصرف أو القرار محل الإعتراض يخص الموظف نفسه. وألا يحمل في طياته صفة الإعتراض الجماعي بأي شكل من
الأشكال.
7 يكون رأي نائب الرئيس للقطاع الذي يتبع له الموظف المعترض هو الرأي النهائي من وجهة نظر القطاع بما لا يتعارض مع نظام
العمل أو لائحة تنظيم العمل بالشركة:؛ ولا يحق له الإعتراض على هذا الرأي لأي جهة اخرى أعلى منهاء وفي حال مخالفة

In [39]:
from langchain_community.document_loaders import PyPDF2
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=250,
    length_function=len
)

# Load the PDF document
pdf_loader = PyPDF2("/Users/nawafalhussain/Desktop/multi_agent_hr/test/data/hr_policy.pdf")
documents = pdf_loader.load()

# Split the documents
splits = text_splitter.split_documents(documents)

# Print how many chunks were created
print(f"Split the documents into {len(splits)} chunks")


ImportError: cannot import name 'PyPDF2' from 'langchain_community.document_loaders' (/Users/nawafalhussain/anaconda3/envs/gpt_agetns/lib/python3.11/site-packages/langchain_community/document_loaders/__init__.py)

In [73]:
splits[0].metadata

{'source': '/Users/nawafalhussain/Desktop/multi_agent_hr/test/data/hr_policy.pdf',
 'file_path': '/Users/nawafalhussain/Desktop/multi_agent_hr/test/data/hr_policy.pdf',
 'page': 0,
 'total_pages': 58,
 'format': 'PDF 1.7',
 'title': '',
 'author': '7',
 'subject': '',
 'keywords': '',
 'creator': 'Microsoft Word',
 'producer': '',
 'creationDate': "D:20250708101456+00'00'",
 'modDate': "D:20250708101456+00'00'",
 'trapped': ''}

In [74]:
splits[0].page_content

'- 1 -\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nذه  ا الدليل قابل للتحديث بناء \n على\nما يطرأ \n من تغيير على سياسات الموارد البشرية \n \n \n \n \n دليل سياســـات \nالموارد البشرية \n \n \nاإلصدار الثالث \n يناير \n5\n202\n م'

# Embeddign

In [75]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

In [76]:
#embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [77]:
# 1. Create an instance
embedding_model = HuggingFaceEmbeddings()

# 2. Embed the documents
document_embeddings = embedding_model.embed_documents([split.page_content for split in splits])

print(f"Created embeddings for {len(document_embeddings)} document chunks")

/var/folders/q9/r2j6s5gj6vqbgwbptm3ztjsc0000gn/T/ipykernel_96785/2735602575.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_model = HuggingFaceEmbeddings()


Created embeddings for 127 document chunks


In [78]:
document_embeddings[0]

[0.015833694487810135,
 0.029529014602303505,
 0.0018292809836566448,
 0.03333534672856331,
 -0.024095473811030388,
 -0.0019795282278209925,
 0.07196268439292908,
 0.027062350884079933,
 0.004122212529182434,
 0.0012764292769134045,
 0.017837433144450188,
 -0.020231712609529495,
 0.07285948842763901,
 -0.01999754272401333,
 0.05803100764751434,
 0.0012254893081262708,
 0.01424502395093441,
 -0.034157998859882355,
 -0.010317913256585598,
 0.012098616920411587,
 -0.030953463166952133,
 0.03234417364001274,
 -0.027719246223568916,
 -0.02606889232993126,
 -0.02087376080453396,
 0.023724811151623726,
 0.009334596805274487,
 -0.029150711372494698,
 -0.03295356407761574,
 -0.015218195505440235,
 0.016606172546744347,
 3.6025812732987106e-05,
 0.0317077711224556,
 -0.034990470856428146,
 2.5376480152772274e-06,
 -0.07061229646205902,
 0.001205307082273066,
 0.0324079804122448,
 -0.0557752288877964,
 0.009073096327483654,
 -0.0008867108845151961,
 -0.020343977957963943,
 -0.028143882751464844,


In [79]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
document_embeddings = embedding_model.embed_documents([split.page_content for split in splits])
print(f"Created embeddings for {len(document_embeddings)} document chunks")


Created embeddings for 127 document chunks


In [80]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document


# Create and persist chrom vectore store

In [81]:

texts = [split.page_content for split in splits]

embedding_model = SentenceTransformerEmbeddings(
    model_name="CAMeL-Lab/bert-base-arabic-camelbert-msa"
)
collection_name = "policy_pdf"


No sentence-transformers model found with name CAMeL-Lab/bert-base-arabic-camelbert-msa. Creating a new one with mean pooling.


In [82]:
# Create and persist the Chroma vector store
db = Chroma.from_texts(texts=texts, embedding=embedding_model,collection_name=collection_name,persist_directory="./new_chroma_db_dpf_arabic")

# Persist to disk
db.persist()

print("✅ Documents embedded and stored in Chroma.")


✅ Documents embedded and stored in Chroma.


In [83]:
query = "give me type of vacations "
search_results = db.similarity_search(query, k=1)

print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 2):
    print(f"Result {i}:")
    print(f"Content: {result.page_content}\n")



Top 2 most relevant chunks for the query: 'give me type of vacations '

Result 2:
Content: - 1 - 
 
 
  
 
 
 
 
 
 
 
 
 
  
 
 
 
 
 
هذا الدليل قابل للتحديث بناء   على  ما يطرأ من تغيير  على سياسات الموارد البشرية 
  
 
 
دليل سياســـات   
الموارد البشرية 
 
 
 
 
الإصدار الثالث  يناير 5202م



In [84]:
retriever = db.as_retriever(search_kwargs={"k":1})
retriever.invoke("give me type of vacations ")

[Document(metadata={}, page_content='- 1 - \n \n \n  \n \n \n \n \n \n \n \n \n \n  \n \n \n \n \n \nهذا الدليل قابل للتحديث بناء   على  ما يطرأ من تغيير  على سياسات الموارد البشرية \n  \n \n \nدليل سياســـات   \nالموارد البشرية \n \n \n \n \nالإصدار الثالث  يناير 5202م')]

In [85]:
from langchain_core.prompts import ChatPromptTemplate


template = """ 
Answer the question based only on the following context:
{context}

You are an experienced HR Specialist responding strictly based on official HR policy documents (retrieved through RAG). Your goals: answer clearly, maintain language consistency, and avoid assumptions.

### ✅ Response Guidelines:
- Base all answers on official HR reference documents.
- Cite documents in square brackets like: [HR_Policy_Name.pdf#page=12]
- If more details are requested, include a reference URL to the HR portal (e.g., https://jusur.channels.com.sa/).
- Provide one-line answers unless the user explicitly asks for more details.
- Respond in the same language as the query (Arabic → Arabic, English → English).
- Use number formatting appropriate for the language.
- If clarification is needed, ask before answering.
- When asked for lists, extract and translate item names exactly as in the source, even if embedded in tables or headings.
- Do not summarize or group items unless instructed. Translate terms directly, preserving their original meaning.
- Never use previous examples unless directly relevant.
- If the source includes violations or procedural notes, do not include these as leave types unless explicitly labeled.

---

### 📋 Structured Response Format (When Details Are Requested):
1. Restate the user's question in more detail, keeping the original language.
2. Policy Overview – Brief summary of the relevant policy.
3. Detailed Explanation – Terms, eligibility, process, and requirements.
4. Exceptions & Special Cases – Role-specific conditions, tenure-based differences.
5. Actionable Steps – Next steps, forms, portals, and approvals.
6. Policy Reference & Support – Cite the policy and provide links or HR contacts if needed.

💡 Use emojis for clarity (📌, 📅, ✅).  
🚫 Avoid dashes (-), hashes (#), or disruptive characters except for #page=12 in citations.

---

### 💬 Examples:
... [your same examples here] ...

---

### 💡 Important Notes:
- Never mix Arabic and English in one answer.
- Never respond with a memorized example unless directly relevant.
- Always follow the actual language of the user input.

---

✅ Your job: give accurate, policy-based HR answers while being language-consistent and user-friendly.

question: {question}

Answer:
"""


prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
llm = OllamaLLM(model="qwen:7b", temperature=0)
rag_chain = (
    {"context":retriever,"question":RunnablePassthrough()}|prompt | llm
)
print(rag_chain.invoke ("h"))

الإجابة حسب السياسات المحددة في 문서人力资源:

"عدد الأيام الإجازية السنوية للمستوى الخامس يعتمد على الشركة والسياسة المحلية. للاطلاع على الرقم الحالي، يجب الرجوع إلى نظام إدارة الموارد البشرية (HRMS) الخاص بشركةك أو الاستفادة من الدعم الفني."

(يرجى تضمين رابط HRMS الخاص بالشركة إذا كان متاحًا.)


In [91]:
def docs2str(docs):
    # docs is a list of Document objects
    return "\n".join([doc.page_content for doc in docs])

from langchain.schema.runnable import RunnableLambda
docs2str = RunnableLambda(docs2str)


In [94]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)
question = "What is the Excellence Award?"
response = rag_chain.invoke(question)
print(response)


The Excellence Award is a recognition program in your organization, designed to honor outstanding achievements and contributions by employees.

Eligibility for the Excellence Award typically depends on meeting specific criteria such as exceptional job performance, innovation, leadership, or overcoming significant challenges.

To receive the Excellence Award, an employee would need to be nominated by their peers or supervisors, and then undergo a review process by a designated committee.

For more detailed information, please refer to your organization's internal policies or HR guidelines.


# Conversation RAG

In [95]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response)
])


In [96]:
chat_history

[HumanMessage(content='What is the Excellence Award?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="The Excellence Award is a recognition program in your organization, designed to honor outstanding achievements and contributions by employees.\n\nEligibility for the Excellence Award typically depends on meeting specific criteria such as exceptional job performance, innovation, leadership, or overcoming significant challenges.\n\nTo receive the Excellence Award, an employee would need to be nominated by their peers or supervisors, and then undergo a review process by a designated committee.\n\nFor more detailed information, please refer to your organization's internal policies or HR guidelines.", additional_kwargs={}, response_metadata={})]

In [97]:

# this will be create a general question about the all question 
#The goal of this contextualization step (often called a “question contextualizer” or “standalone question generator”) 
# is to make your RAG or chatbot system smarter with follow-up questions in multi-turn conversations.


from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

# Define the system prompt
contextualize_q_system_prompt = (
    "Given a chat and the latest user question, "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do not answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

# Create the chat prompt template
contextualize_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

# Create the chain: prompt -> LLM -> output parser
contextualize_chain = contextualize_prompt | llm | StrOutputParser()

# Invoke the chain with the proper keys
result = contextualize_chain.invoke({"input": "how can I get it?","chat_history": chat_history})

print(result)


To obtain the Excellence Award in your organization, you would typically follow these steps:

1. **Meet the Criteria**: Understand the eligibility criteria mentioned earlier. This includes exceptional job performance, innovation, leadership, overcoming challenges, and being nominated by peers or supervisors.

2. **Nomination Process**: If you meet the criteria, gather information about your accomplishments and prepare a nomination form. Submit this form to your supervisor or a designated person within your organization.

3. **Review Process**: After submitting your nomination, it will be reviewed by a committee or panel of experts in your field. They will assess your achievements against the established criteria.

4. **Notification**: If you are selected for the Excellence Award, you will receive notification from your organization. This may include an official ceremony or recognition event.

Remember that the process and timeline can vary depending on your organization's policies and 

In [98]:
from langchain.chains import create_history_aware_retriever
history_aware_retriever = create_history_aware_retriever(
    llm,retriever,contextualize_prompt
)
history_aware_retriever.invoke({"input": "how can I get it?", "chat_history": chat_history})


[Document(metadata={}, page_content='1.4.4.5.1 يت حال استمرار يترة الإعارة حتى اخر90 يوما  من دورة الأداء يتن تطييم  بناء  على أدائ  يت الشر ة التابعة. \n1.4.4.5.2 يت حال انت اء يترة ا عارة قبل ال90 يوما  الأخيرة من دورة الأداء يتن تطييم  بناء  على أدائ  يت م ام  الرئيسرة من قبل المدير\nالمباشر وعلى المدير المباشر تقمين أداء الموي  يت الشر ات التابعة قمن التطيين الن ائت. \n1.4.4.5.3 يت حال انت اء إعارة الموي  لشرررر ة تابعة وتبع ا انتطال للشرررر ة التابعة و انت المدة بين ا اقل من90 يوما  تطويميا  يتن قرررن\nالمدة بين الإعارة وا نتطال. \n1.4.5 يتن تطيين الموييون المعارون لأغراض تطويري  أو ابتعاث أ اديمت حس  القوابط التالية: \n1.4.5.1 يت حال  ان مدة الإعارة التطويرية او ا بتعاث لمدة مسررراوية أو ت يد عن180 يوما  تطويميا ؛ يتن تطيين أدائ ن بناء على نتائج ن\nيت البرنامج  أو ا بتعاث. \n1.4.5.2 يت حرال  رانرت مردة الإعرارة التطويريرة او ا بتعراث تطرل عن180  يومرا  تطويميرا ؛ يتن تطيين أدائ نبنراء على أدائ ن يت م رام ن \nالرئيسية من قبل المدير المباشر وعلى المدير المباشر تقمين أداء الموي  يت 

In [99]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

qa_promot = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful AI assistenct. Use the following context to answer the user's question "),
    ("system", "context : {context}"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_promot)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [100]:
rag_chain.invoke({"input":"how can I get it?","chat_history":chat_history})

{'input': 'how can I get it?',
 'chat_history': [HumanMessage(content='What is the Excellence Award?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The Excellence Award is a recognition program in your organization, designed to honor outstanding achievements and contributions by employees.\n\nEligibility for the Excellence Award typically depends on meeting specific criteria such as exceptional job performance, innovation, leadership, or overcoming significant challenges.\n\nTo receive the Excellence Award, an employee would need to be nominated by their peers or supervisors, and then undergo a review process by a designated committee.\n\nFor more detailed information, please refer to your organization's internal policies or HR guidelines.", additional_kwargs={}, response_metadata={})],
 'context': [Document(metadata={}, page_content='1.4.4.5.1 يت حال استمرار يترة الإعارة حتى اخر90 يوما  من دورة الأداء يتن تطييم  بناء  على أدائ  يت الشر ة التابعة. \n1.4.4.5.2 يت حا

# Using the History-Aware RAG Chain

In [101]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
question1 = "how many type of leaves?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

print(f"Human: {question1}")
print(f"AI: {answer1}\n")

question2 = "list it to me"
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=answer2)
])

print(f"Human: {question2}")
print(f"AI: {answer2}")


Human: how many type of leaves?
AI: There are countless types of leaves, as plants can have a wide variety of foliage structures. The number is difficult to quantify precisely, but it's in the millions or even higher. Each plant species may have its own unique set of leaf shapes and sizes.

Human: list it to me
AI: Sure, I'll list the information you provided:

1. Context: - 39 (not clear what this refers to)
- A program or initiative related to housing support and credit loans

2. Points for calculating points:
- Mechanism for assigning points based on performance or achievement
- Points are allocated considering different categories or criteria

3. General rules or provisions:
- Directives or regulations governing the housing support program
- Rules about eligibility, loan terms, and repayment options
- Guidelines for monitoring and evaluating the performance of the applicants or recipients

4. Specific examples or details:
- Examples of solutions or products included in the housing 

# Building a Multi-User Chatbot with SQLite Storage


In [102]:
import sqlite3
from datetime import datetime
import uuid

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    user_query TEXT,
    gpt_response TEXT,
    model TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()


In [ ]:
# Example usage for a new user
session_id = str(uuid.uuid4())
question = "What is GreenGrow Innovations?"
chat_history = get_chat_history(session_id)
answer = rag_chain.invoke({"input": question, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question, answer, "gpt-3.5-turbo")
print(f"Human: {question}")
print(f"AI: {answer}\n")
print("Session ID:", session_id)



Inserting: 1b73e667-8fda-4a32-bd83-ab510f607a41, What is GreenGrow Innovations?, GreenGrow Innovations is not a widely recognized or well-known company. It could be a hypothetical or regional organization focused on innovation in environmental or agricultural sectors.

Without specific information, it's difficult to provide detailed details about the company. If you have more context or if this is a real company but not very well known, please provide that information so I can assist you better., gpt-3.5-turbo
Human: What is GreenGrow Innovations?
AI: GreenGrow Innovations is not a widely recognized or well-known company. It could be a hypothetical or regional organization focused on innovation in environmental or agricultural sectors.

Without specific information, it's difficult to provide detailed details about the company. If you have more context or if this is a real company but not very well known, please provide that information so I can assist you better.

Inserting: 1b73e667-8

In [ ]:
# Example of a follow-up question
question2 = "What was their first product?"
chat_history = get_chat_history(session_id)
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "gpt-3.5-turbo")
print(f"Human: {question2}")
print(f"AI: {answer2}")

In [108]:
def insert_application_logs(session_id, user_query, gpt_response, model):
    print(f"Inserting: {session_id}, {user_query}, {gpt_response}, {model}")
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()


In [109]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("rag_app.db")
df = pd.read_sql_query("SELECT * FROM application_logs", conn)
print(df.head())  # Display the first few rows
conn.close()


   id                            session_id                      user_query  \
0   1  31d814b9-20fb-44b5-85a0-e68a2074190b  What is GreenGrow Innovations?   
1   2  31d814b9-20fb-44b5-85a0-e68a2074190b   What was their first product?   
2   3  1b73e667-8fda-4a32-bd83-ab510f607a41  What is GreenGrow Innovations?   
3   4  1b73e667-8fda-4a32-bd83-ab510f607a41   What was their first product?   

                                        gpt_response          model  \
0  GreenGrow Innovations is not a widely recogniz...  gpt-3.5-turbo   
1  The context provided does not specify which co...  gpt-3.5-turbo   
2  GreenGrow Innovations is not a widely recogniz...  gpt-3.5-turbo   
3  The context provided does not specify which co...  gpt-3.5-turbo   

            created_at  
0  2025-07-14 11:18:45  
1  2025-07-14 11:19:05  
2  2025-07-14 11:23:02  
3  2025-07-14 11:23:30  
